In [2]:
import datetime
from datetime import date
import pandas as pd
import requests
import config

## Extract

In [3]:
event_editions_url = "https://dashboard.activcreew.app/api/event-editions?fields=name&populate[address][fields]=country,city&populate[event][fields]=id&populate[date_table][fields]=begin, end"

response_event_editions = requests.get(event_editions_url, headers=config.my_headers)
event_editions_data = response_event_editions.json()["data"]
df_event_editions = pd.DataFrame(event_editions_data)
df_event_editions

,id,documentId,name,address,event,date_table
0,6,escwtfkm1a1sd0di8mrzxgtk,Marathon du lac d'Annecy 2025,"{'id': 1222, 'documentId': 'fjho8gvecwprol9v0m...","{'id': 6, 'documentId': 'h4e8nx3mnj9fmd22983hg...","{'id': 481, 'documentId': 'ucvzbe5fj2nq0vzutl4..."


## Transform

In [4]:
# Get fields from columns dicts
df_event_editions["country"] = df_event_editions["address"].apply(lambda x: x.get("country"))
df_event_editions["city"] = df_event_editions["address"].apply(lambda x: x.get("city"))
df_event_editions["event_id"] = df_event_editions["event"].apply(lambda x: x.get("id"))
df_event_editions["begin_event"] = df_event_editions["date_table"].apply(lambda x: x.get("begin"))
df_event_editions["end_event"] = df_event_editions["date_table"].apply(lambda x: x.get("end"))
df_event_editions

,id,documentId,name,address,event,date_table,country,city,event_id,begin_event,end_event
0,6,escwtfkm1a1sd0di8mrzxgtk,Marathon du lac d'Annecy 2025,"{'id': 1222, 'documentId': 'fjho8gvecwprol9v0m...","{'id': 6, 'documentId': 'h4e8nx3mnj9fmd22983hg...","{'id': 481, 'documentId': 'ucvzbe5fj2nq0vzutl4...",FR,Annecy,6,2025-04-25T03:00:00.000Z,2025-04-27T18:00:00.000Z


In [5]:
# Delete useless columns
df_event_editions.drop(columns=["date_table", 'documentId', 'address', 'event'], inplace=True)
df_event_editions

,id,name,country,city,event_id,begin_event,end_event
0,6,Marathon du lac d'Annecy 2025,FR,Annecy,6,2025-04-25T03:00:00.000Z,2025-04-27T18:00:00.000Z


In [6]:
# Convert dates to datetime and format output
df_event_editions["begin_event"] = pd.to_datetime(df_event_editions["begin_event"])
df_event_editions["end_event"] = pd.to_datetime(df_event_editions["end_event"])

# Get year, month, number of days
df_event_editions["year"] = df_event_editions["begin_event"].dt.year
df_event_editions["month"] = df_event_editions["begin_event"].dt.month
df_event_editions["number of days"] = (df_event_editions["end_event"] - df_event_editions["begin_event"]).dt.days

df_event_editions

,id,name,country,city,event_id,begin_event,end_event,year,month,number of days
0,6,Marathon du lac d'Annecy 2025,FR,Annecy,6,2025-04-25 03:00:00+00:00,2025-04-27 18:00:00+00:00,2025,4,2


## Load

In [7]:
df_event_editions.to_csv('csv_files/event_editions_data.csv', index=False)